In [1]:
import numpy as np
import pandas as pd
import pyodbc
import sqlalchemy
import sqlite3
from subprocess import check_output
import os

%sql sqlite://

'Connected: @None'

In [2]:
summer = pd.read_csv('/kaggle/input/data-sql/summer.csv')
summer.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,MALOKINIS Ioannis,GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,CHASAPIS Spiridon,GRE,Men,100M Freestyle For Sailors,Silver


In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////summers', echo=False)
summer.to_sql('Summer_Medals', con = engine)

# PRACTICEs
## 1. Pivoting
### 1.1. Transforming table.
`Before`: **`gold medals`** awards to 3 countries : `USA, Russia` and `China_dog`

| Country | Year | gold_medals  |
|---------|------|--------------|
| CHN     | 2008 | 74           |
| CHN     | 2012 | 56           |
| RUS     | 2008 | 43           |
| RUS     | 2012 | 47           |
| USA     | 2008 | 125          |
| USA     | 2012 | 147          |

`After`: **Pivoted by `Year`**; this way make easier to scan, especially if pivoted by a `choronologically ordered column`.

| Country | 2008 | 2012  |
|---------|------|-------|
| CHN     |  74  |  56   |
| RUS     |  43  |  47   |
| USA     | 125  | 147   |


### 1.2. How to implement? Using `CROSSTAB`

#### Syntax:

        CREATE EXTENSION IF NOT EXISTS table_function;

        SELECT * FROM CROSSTAB( $$          
               source_sql_text(SELECT , 
                               FROM, WHERE / GROUP BY ...
                               $$ ) AS ct( column_1  DATA_TYPE_1,
                                           column_2  DATA_TYPE_2,
                                               ... ,
                                           column_n  DATA_TYPE_n
                                           )
        ORDER BY something
Here, the `DATA_TYPE` can be `"VARCHAR, INTEGER"`,...

Firstly, I will simulate the preceding tables in section 1.1.

In [4]:
pd.read_sql(
    """ 
        SELECT year, country, COUNT(*) AS gold_medals
        FROM Summer_medals
        WHERE (medal = 'Gold') AND
             (country IN ('CHN', 'RUS', 'USA')) AND (year IN (2008, 2012))
        GROUP BY country, year
    """, con = engine)

,Year,Country,gold_medals
0,2008,CHN,74
1,2012,CHN,56
2,2008,RUS,43
3,2012,RUS,47
4,2008,USA,125
5,2012,USA,147


Then, using `crosstab`

In [5]:
pd.read_sql(
    """ 
    CREATE EXTENSION IF NOT EXISTS tablefunc;
    SELECT * FROM CROSSTAB($$
                                SELECT country, year, COUNT(*) :: INTEGER AS gold_medals
                                FROM Summer_medals
                                WHERE (medal = 'Gold') AND
                                     (country IN ('CHN', 'RUS', 'USA')) AND (year IN (2008, 2012))
                                GROUP BY country, year
                          $$) AS ct(
                                    country VARCHAR,
                                    "2008" INTEGER,
                                    "2012" INTEGER
                                    )
        """, con = engine)

,country,2008,2012
0,CHN,74,56
1,RUS,43,47
2,USA,125,147


### Exercise 1.1. Basic pivot

You have the following table of Pole Vault gold medalist countries by gender in 2008 and 2012.

| Gender | Year | Country |
|--------|------|---------|
| Men    | 2008 | AUS     |
| Men    | 2012 | FRA     |
| Women  | 2008 | RUS     |
| Women  | 2012 | USA     |

Pivot it by Year to get the following reshaped, cleaner table.

| Gender | 2008 | 2012 |
|--------|------|------|
| Men    | AUS  | FRA  |
| Women  | RUS  | USA  |

In [6]:
pd.read_sql(
    """
        -- Create the correct extention to enable CROSSTAB
        CREATE EXTENSION IF NOT EXISTS tablefunc;

        SELECT * FROM CROSSTAB($$
          SELECT
                Gender, Year, Country
          FROM Summer_Medals
          WHERE year IN (2008, 2012)
                AND medal = 'Gold'
                AND event = 'Pole Vault'
          ORDER By gender ASC,year ASC;
          
        -- Fill in the correct column names for the pivoted table
        $$) AS ct (gender VARCHAR,
                   "2008" VARCHAR,
                   "2012" VARCHAR)

        ORDER BY gender ASC;    
    """, con = engine)

,gender,2008,2012
0,Men,AUS,FRA
1,Women,RUS,USA


### Exercise 1.2. Pivoting with ranking
You want to produce an easy scannable table of the rankings of the three most populous EU countries by how many gold medals they've earned in the 2004 through 2012 Olympic games. The table needs to be in this format:

| Country | 2004 | 2008 | 2012 |
|---------|------|------|------|
| FRA     | ...  | ...  | ...  |
| GBR     | ...  | ...  | ...  |
| GER     | ...  | ...  | ...  |

You'll need to count the gold medals each country has earned, produce the ranks of each country by medals earned, then pivot the table to this shape.

#### Instructions
**Step 1.**Count the gold medals that France (FRA), the UK (GBR), and Germany (GER) have earned per country and year.

In [7]:
pd.read_sql(
    """
        -- Count the gold medals per country and year
        SELECT country, year,
                COUNT(*) AS Awards
        FROM Summer_Medals
        WHERE country IN ('FRA', 'GBR', 'GER')
          AND year IN (2004, 2008, 2012)
          AND medal = 'Gold'
        GROUP BY country, year
        ORDER BY country ASC, year ASC
    """, con = engine)

,country,year,awards
0,FRA,2004,21
1,FRA,2008,25
2,FRA,2012,30
3,GBR,2004,17
4,GBR,2008,31
5,GBR,2012,48
6,GER,2004,41
7,GER,2008,42
8,GER,2012,45


**Step 2.** Select the country and year columns, then rank the three countries by how many gold medals they earned per year.

In [8]:
pd.read_sql(
    """
        WITH Country_Awards AS ( SELECT country, year,
                                         COUNT(*) AS Awards
                                  FROM Summer_Medals
                                  WHERE country IN ('FRA', 'GBR', 'GER')
                                    AND Year IN (2004, 2008, 2012)
                                    AND Medal = 'Gold'
                                  GROUP BY Country, Year)

        SELECT country, year,
            -- Rank by gold medals earned per year
            RANK() OVER(PARTITION BY year ORDER BY awards) :: INTEGER AS rank
        FROM Country_Awards
        ORDER BY Country ASC, Year ASC;
    """, con = engine)

,country,year,rank
0,FRA,2004,2
1,FRA,2008,1
2,FRA,2012,1
3,GBR,2004,1
4,GBR,2008,2
5,GBR,2012,3
6,GER,2004,3
7,GER,2008,3
8,GER,2012,2


**Step 3.** Pivot the query's results by Year by filling in the new table's correct column names.

In [9]:
pd.read_sql(
    """
        CREATE EXTENSION IF NOT EXISTS tablefunc;

        SELECT * FROM CROSSTAB($$
          WITH Country_Awards AS ( SELECT country, year,
                                          COUNT(*) AS Awards
                                    FROM Summer_Medals
                                    WHERE country IN ('FRA', 'GBR', 'GER')
                                      AND Year IN (2004, 2008, 2012)
                                      AND Medal = 'Gold'
                                    GROUP BY Country, Year)

                  SELECT country, year,
                        RANK() OVER (PARTITION BY Year
                                       ORDER BY Awards DESC) :: INTEGER AS rank
                  FROM Country_Awards
                  ORDER BY Country ASC, Year ASC;

                -- Fill in the correct column names for the pivoted table
                                $$) AS ct (country VARCHAR,
                                           "2004" INTEGER,
                                           "2008" INTEGER,
                                           "2012" INTEGER)

        Order by Country ASC;
    """, con = engine)

,country,2004,2008,2012
0,FRA,2,3,3
1,GBR,3,2,1
2,GER,1,1,2


## 2. ROLLUP and CUBE
#### Group-level total.

For example: Chinese and Russian medals in the 2008 Summer Olympic per medal class by using **the old way**

In [10]:
pd.read_sql(
    """ 
        SELECT country, medal, COUNT(*) AS adwards
        FROM Summer_Medals
        WHERE year = 2008 AND country IN ('CHN', 'RUS')
        GROUP BY country, medal
        ORDER BY country ASC, medal ASC
    """, con = engine)

,Country,Medal,adwards
0,CHN,Bronze,57
1,CHN,Gold,74
2,CHN,Silver,53
3,RUS,Bronze,56
4,RUS,Gold,43
5,RUS,Silver,44


and using **ROLLUP**

            SELECT country, medal, COUNT(*) AS adwards
            FROM Summer_Medals
            WHERE year = 2008 AND country IN ('CHN', 'RUS')
            GROUP BY country, ROLLUP(medal)
            ORDER BY country ASC, medal ASC;
So,

| country |	medal |	adwards|
|---------|-------|--------|
| CHN     |	Bronze|	57     |
| CHN	  | Gold  |	74     |
| CHN	  | Silver|	53     |
| CHN	  | null  |	184    |
| RUS	  | Bronze|	56     |
| RUS	  | Gold  |	43     |
| RUS	  | Silver|	44     |
| RUS	  | null  |	143    |

**Comments :**
> **`ROLLUP`** is a **`GROUP BY`** subclause that includes extra rows for **group-level-aggregations**
>
> **`GROUP BY`** `country,`**`ROLLUP(medal)`** will count all `country` and `medal-`level totals then count only `country-`level totals and fill in `medal` with `null or NaN` values for these rows.

**ROLLUP query**

In [11]:
pd.read_sql(
    """ 
            SELECT country, medal, COUNT(*) AS adwards
            FROM Summer_Medals
            WHERE year = 2008 AND country IN ('CHN', 'RUS')
            GROUP BY ROLLUP(country, medal)
            ORDER BY country ASC, medal ASC;    
    """, con = engine)

,country,medal,adwards
0,CHN,Bronze,57
1,CHN,Gold,74
2,CHN,Silver,53
3,CHN,NaN,184
4,RUS,Bronze,56
5,RUS,Gold,43
6,RUS,Silver,44
7,RUS,NaN,143
8,NaN,NaN,327


#### Comments
> **`ROLLUP`** is hierarchical, de-aggregating from the `left-most` provided column to the `right-most`
>
>> **`ROLLUP(country, medal)`** includes `country-level` totals
>
>> **`ROLLUP(medal, country)`** includes `medal-level` totals

=======================================

#### Using `CUBE in `GROUP BY`.

            SELECT country, medal, COUNT(*) AS adwards
            FROM Summer_Medals
            WHERE year = 2008 AND country IN ('CHN', 'RUS')
            GROUP BY CUBE(country, medal)
            ORDER BY country ASC, medal ASC;

| country |	medal   | adwards |
|---------|---------|---------|
| CHN     |	Bronze  |  	 57   |
| CHN	  | Gold	|    74   |
| CHN	  | Silver	|    53   |
| CHN	  | null	|   184   |
| RUS	  | Bronze	|    56   |
| RUS	  | Gold	|    43   |
| RUS	  | Silver	|    44   |
| RUS	  | null	|   143   |
| null	  | Bronze	|   113   |
| null	  | Gold	|   117   |
| null	  | Silver	|    97   |
| null	  | null	|   327   |


**Comments**
> **`CUBE`** is non-hierarchical of **`ROLLUP`**
>
> It generates all possibles `group-level-aggregation`. For example; the syntax `CUBE(country, medal)` count the `country-level`, `medal-level` and the `grand-totals`
>
> Remember that the `medal-level-totals` are included

=========================================

#### Compares: ROLLUP vs CUBE by the following examples
**Source_given**

| Year | Quarter | Sales |
|------|---------|-------|
| 2008 |  Q1     | 12   |
| 2008 |  Q2     | 15   |
| 2009 |  Q1     | 21   |
| 2009 |  Q2     | 27   |

#### A. Using `ROLLUP(year, quarter)`
We obtain

| Year | Quarter | Sales |
|------|---------|-------|
| 2008 |  null   | 27    |
| 2008 |  null   | 48    |
| null |  null   | 75    |

#### B. Using `CUBE(year, quarter)`
Then,

| Year | Quarter | Sales |
|------|---------|-------|
| null |  Q1     | 33    |
| null |  Q2     | 42    |

#### C. Remarks when using `ROLLUP` and `CUBE`
> We use **ROLLUP** when we have **`hierarchical data (e.g, date parts)`** and **don't want all possible `group-level-aggregations`**
>
> We use **CUBE** when we **want all possible `group-level-aggregations`**

### Exercies 2.1. Country-level subtotals

You want to look at three `Scandinavian countries`' earned `gold medals` per `country` and `gender` in the year 2000. You're also interested in Country-level subtotals to get the total medals earned for each `country`, but `Gender`-level subtotals don't make much sense in this case, so disregard them.

#### Instructions
Count the `gold medals` awarded per `country` and `gender`.

Generate `Country`-level gold award counts.

In [12]:
pd.read_sql(
    """ 
        -- Count the gold medals per country and gender
        SELECT country, gender,
               COUNT(*) AS Gold_Awards
        FROM Summer_Medals
        WHERE year = 2004
          AND Medal = 'Gold'
          AND Country IN ('DEN', 'NOR', 'SWE')
        -- Generate Country-level subtotals
        GROUP BY country, ROLLUP(gender)
        ORDER BY country ASC, gender ASC;    
    """, con = engine)

,country,gender,gold_awards
0,DEN,Men,4
1,DEN,Women,15
2,DEN,NaN,19
3,NOR,Men,3
4,NOR,Women,2
5,NOR,NaN,5
6,SWE,Men,4
7,SWE,Women,1
8,SWE,NaN,5


### Exercise 2.2. All group-level subtotals
You want to break down all medals awarded to Russia in the 2012 Olympic games per gender and medal type. Since the medals all belong to one country, Russia, it makes sense to generate all possible subtotals (Gender- and Medal-level subtotals), as well as a grand total.

Generate a breakdown of the medals awarded to Russia per country and medal type, including all group-level subtotals and a grand total.

#### Instructions
Count the medals awarded per gender and medal type.
Generate all possible group-level counts (per gender and medal type subtotals and the grand total).

In [13]:
pd.read_sql(
    """ 
        -- Count the medals per country and medal type
        SELECT gender, medal,
                COUNT(*) AS Awards
        FROM Summer_Medals
        WHERE
          Year = 2012
          AND Country = 'RUS'
        -- Get all possible group-level subtotals
        GROUP BY CUBE(gender, medal)
        ORDER BY Gender ASC, Medal ASC;    
    """, con = engine)

,gender,medal,awards
0,Men,Bronze,34
1,Men,Gold,23
2,Men,Silver,7
3,Men,NaN,64
4,Women,Bronze,17
5,Women,Gold,24
6,Women,Silver,25
7,Women,NaN,66
8,NaN,Bronze,51
9,NaN,Gold,47


## 3. A survey of useful functions
### 3.1 Null ahoy
For example, as discuss in the previous section, using ROLLUP will return the `null-values`

            SELECT country, medal, COUNT(*) AS adwards
            FROM Summer_Medals
            WHERE year = 2008 AND country IN ('CHN', 'RUS')
            GROUP BY country, ROLLUP(medal)
            ORDER BY country ASC, medal ASC;
then,

| country |	medal |	adwards|
|---------|-------|--------|
| CHN     |	Bronze|	57     |
| CHN	  | Gold  |	74     |
| CHN	  | Silver|	53     |
| CHN	  | null  |	184    |
| RUS	  | Bronze|	56     |
| RUS	  | Gold  |	43     |
| RUS	  | Silver|	44     |
| RUS	  | null  |	143    |

#### Solve: COALESCE
> This takes a list of values and returns the first non-null value, going from left to right.
>
> Useful when using `SQL operations` that returns `null` such as:
>
>> **LEAD** and  **LAG**
>
>> **CUBE** and **ROLLUP**
>
>> pivoting function

For example, 

            SELECT COALESCE(country, 'both country') AS country, 
                   COALESCE( medal, 'all_medal') AS medals, 
                    COUNT(*) AS adwards
            FROM Summer_Medals
            WHERE year = 2008 AND country IN ('CHN', 'RUS')
            GROUP BY country, ROLLUP(medal)
            ORDER BY country ASC, medal ASC;
then, we will get the result

| country |	medal      | adwards|
|---------|------------|--------|
| CHN     |	Bronze     |	57  |
| CHN	  | Gold       |	74  |
| CHN	  | Silver     |	53  |
| CHN	  | all_medal  |	184 |
| RUS	  | Bronze     |	56  |
| RUS	  | Gold       |	43  |
| RUS	  | Silver     |	44  |
| RUS	  | all_medal  |	143 |

Likewise, using **ROLLUP(country, medal)** returns

In [14]:
pd.read_sql(
    """ 
            SELECT COALESCE(country, 'both country') AS country, 
                   COALESCE( medal, 'all_medal') AS medals, 
                    COUNT(*) AS adwards
            FROM Summer_Medals
            WHERE year = 2008 AND country IN ('CHN', 'RUS')
            GROUP BY ROLLUP(country, medal)
            ORDER BY country ASC, medal ASC;    
    """, con = engine)

,country,medals,adwards
0,CHN,Bronze,57
1,CHN,Gold,74
2,CHN,Silver,53
3,CHN,all_medal,184
4,RUS,Bronze,56
5,RUS,Gold,43
6,RUS,Silver,44
7,RUS,all_medal,143
8,both country,all_medal,327


#### COMPRESSING DATA
Assume that we have (@):

| country | rank |
|---------|------|
| USA     |   1  |
| RUS     |   2  |
| GER     |   3  

> **`Rank`** is redundant because the ranking is implied.

and we want to compress it into (@@)

            USA,    RUS,     GER

> Succint and provides all information needed because the ranking is implied

**Using `STRING_AGG(column, seperator)`** takes all the values of the column and concatenates them with the seperator in between each values. For example, using

    STRING_AGG(country, ',')
to compress (@) into (@@)

Now, trying with the following code; here is (@)

In [15]:
pd.read_sql(
    """ 
        WITH countries_medal AS ( SELECT country, COUNT(*) AS medal
                            FROM Summer_Medals
                            WHERE year = 2012 AND country IN ('USA', 'RUS', 'GER')
                            GROUP BY country
                            )
        SELECT country,
               RANK() OVER(ORDER BY medal DESC) AS rank
        FROM countries_medal
    """, con = engine)

,country,rank
0,USA,1
1,RUS,2
2,GER,3


and (@@)

In [16]:
pd.read_sql(
    """ 
        WITH countries_medal AS ( SELECT country, COUNT(*) AS medal
                            FROM Summer_Medals
                            WHERE year = 2012 AND country IN ('USA', 'RUS', 'GER')
                            GROUP BY country
                            ),
             countries_rank AS ( SELECT country, 
                                 RANK() OVER(ORDER BY medal DESC) AS rank
                                 FROM countries_medal
                            ) 
        SELECT STRING_AGG(country, ', ')
        FROM countries_medal
    """, con = engine)

USA, RUS, GER


### Exercise 3.1. Cleaning up results
Returning to the breakdown of Scandinavian awards you previously made, you want to clean up the results by replacing the nulls with meaningful text.

#### Instructions
Turn the nulls in the Country column to All countries, and the nulls in the Gender column to All genders.

In [17]:
pd.read_sql(
    """
        SELECT
              COALESCE(Country, 'All countries') AS Country,
              COALESCE(Gender, 'All genders') AS Gender,
              COUNT(*) AS Awards
        FROM Summer_Medals        
        WHERE (year = 2004) AND (medal = 'Gold')
              AND Country IN ('DEN', 'NOR', 'SWE')
        
        GROUP BY ROLLUP(country, gender)
        
        ORDER BY country ASC, gender ASC;    
    """, con = engine)

,country,gender,awards
0,All countries,All genders,29
1,DEN,All genders,19
2,DEN,Men,4
3,DEN,Women,15
4,NOR,All genders,5
5,NOR,Men,3
6,NOR,Women,2
7,SWE,All genders,5
8,SWE,Men,4
9,SWE,Women,1


### Exercise 3.2. Summarizing results
After ranking each country in the 2000 Olympics by gold medals awarded, you want to return the top 3 countries in one row, as a comma-separated string. In other words, turn this:

| Country | Rank |
|---------|------|
| USA     | 1    |
| RUS     | 2    |
| AUS     | 3    |
| ...     | ...  |

into this:

            USA, RUS, AUS
#### Instructions 
**Step 1.** Rank countries by the medals they've been awarded.

In [18]:
pd.read_sql(
    """ 
        WITH Country_Medals AS (SELECT country,
                                        COUNT(*) AS Medals
                                  FROM Summer_Medals
                                  WHERE Year = 2000
                                    AND Medal = 'Gold'
                                  GROUP BY Country)
          SELECT
                country,
                RANK() OVER(ORDER BY medals DESC) AS Rank
          FROM Country_Medals
          ORDER BY Rank ASC;
    """, con = engine)

,country,Rank
0,USA,1
1,RUS,2
2,AUS,3
3,CHN,4
4,GER,5
5,NED,6
6,ROU,6
7,HUN,8
8,CUB,9
9,FRA,9


**Step 2.** Return the top 3 countries by medals awarded as one comma-separated string.

In [19]:
pd.read_sql(
    """
        WITH Country_Medals AS (
          SELECT
            Country,
            COUNT(*) AS Medals
          FROM Summer_Medals
          WHERE Year = 2000
            AND Medal = 'Gold'
          GROUP BY Country),

          Country_Ranks AS (
          SELECT
            Country,
            RANK() OVER (ORDER BY Medals DESC) AS Rank
          FROM Country_Medals
          ORDER BY Rank ASC)

        -- Compress the countries column
        SELECT STRING_AGG(country, ', ')
        FROM Country_Ranks
        -- Select only the top three ranks
        WHERE rank <= 3;    
    """, con = engine)

USA, RUS, AUS
